In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
rng = RandomState(39482014)
plt.plot(np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8)))
a = np.sin(np.linspace(0, 2*np.pi, endpoint=False, num=8))
amps_a = np.abs(rng.randn(320) + 2)
phases_a = rng.rand(320) * 2 * np.pi
signals = np.array([amp * np.sin(np.linspace(0, 2*np.pi, endpoint=True, num=8) + phase) 
     for amp, phase in zip(amps_a, phases_a)])
plt.figure()
plt.plot(signals[:10].squeeze().T)
inputs = [np_to_var(signals[:,None,:,None], dtype=np.float32).cuda()]

In [ ]:
from matplotlib.lines import Line2D
plt.figure(figsize=(10,6))
for i_class in range(len(inputs)):
    plt.plot(var_to_np(inputs[i_class].squeeze()).T, color=seaborn.color_palette()[i_class],lw=0.5);
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
plt.legend(lines, ['Right', 'Rest',], bbox_to_anchor=(1,1,0,0))
plt.title('Input signals')

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DSFTransform
from ampphase import outs_to_amp_phase


set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    rev_block(4,32),
    rev_block(4,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    rev_block(8,32),
    rev_block(8,32),
    ViewAs((-1,8,1,1), (-1,8))
)
feature_model.cuda()




from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    nn.Conv2d(1,16, (3,1), stride=1, padding=(1,0),bias=True),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    nn.AvgPool2d((2,1)),
    res_block(16,32),
    res_block(16,32),
    ViewAs((-1,16,2,1), (-1,32)),
    )


adv_model = ProjectionDiscriminator(adv_model,32,2,)

adv_model.cuda()


from reversible.training import hard_init_std_mean
n_dims = inputs[0].shape[2]
n_clusters = 1
means_per_cluster = [th.autograd.Variable(th.ones(n_dims).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims).cuda(), requires_grad=True)
                    for _ in range(n_clusters)]

super_means_per_cluster = [th.autograd.Variable(th.ones(2).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
super_stds_per_cluster = [th.autograd.Variable(th.zeros(2).cuda(), requires_grad=True)
                     for _ in range(n_clusters)]
for i_class in range(n_clusters):
    this_outs = feature_model(inputs[i_class])
    amps, phases = outs_to_amp_phase(this_outs)

    means_per_cluster[i_class].data[:amps.shape[1]] = th.mean(amps, dim=0).data - th.mean(amps).data
    #stds_per_cluster[i_class].data[:amps.shape[1]] = th.log(th.std(amps, dim=0).data / th.std(amps.data))
    stds_per_cluster[i_class].data[:amps.shape[1]] = -3#th.log(th.std(amps, dim=0).data / th.std(amps.data))
    # override phase
    means_per_cluster[i_class].data[amps.shape[1]:] = 0
    stds_per_cluster[i_class].data[amps.shape[1]:] = -3#0




from copy import deepcopy
optimizer = th.optim.Adam(
                          [
    {'params': list(feature_model.parameters()),
    'lr': 1e-3,
    'weight_decay': 0},], betas=(0,0.9))

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster + super_means_per_cluster + super_stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))

optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-3, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))

In [ ]:

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from ampphase import (amp_phase_to_x_y, get_amp_phase_samples, amp_phase_sample_to_x_y,
                      outs_to_amp_phase, to_amp_phase, get_amp_phase_xy_samples,
                     switch_to_other_class)

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan
gradient_loss = np_to_var([np.nan],dtype=np.float32)

In [ ]:
def invert_hierarchical(features):
    return invert(feature_model, features)

In [ ]:
def get_samples(n_samples, i_class):
    super_samples = get_gauss_samples(n_samples, super_means_per_cluster[i_class],
                                     super_stds_per_cluster[i_class], truncate_to=3)
    mean = means_per_cluster[i_class]
    mean = th.cat((mean[:len(mean)//2] - th.mean(mean[:len(mean)//2]),
                  mean[len(mean)//2:] - th.mean(mean[len(mean)//2:])))
    log_std = stds_per_cluster[i_class]
    std = th.exp(log_std)

    subsamples = get_gauss_samples(n_samples, mean, std, truncate_to=3)
    super_samples = super_samples.view(super_samples.shape[0], super_samples.shape[1], 1).repeat(
        1,1,subsamples.shape[1] // 2).view(super_samples.shape[0], -1)
    samples = (super_samples + subsamples)
    return samples

In [ ]:
n_epochs = 1001
rng = RandomState(349384)
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_adv.zero_grad()
    optimizer.zero_grad()
    optim_dist.zero_grad()
    i_class = 0
    this_inputs = inputs[i_class] 
    mean = means_per_cluster[i_class]
    log_std = stds_per_cluster[i_class]
    std = th.exp(log_std)
    n_samples = len(this_inputs)
    samples = get_samples(n_samples, i_class)
    inverted = invert_hierarchical(samples)
    y = np_to_var([i_class]).cuda()
    score_fake = adv_model(inverted, y)
    if (i_epoch % 10) != 0:
        score_real = adv_model(this_inputs, y)
        gradient_loss = gradient_penalty(adv_model, this_inputs, inverted[:(len(this_inputs))], y)
        d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 10
        d_loss.backward()
        d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
    else:
        g_loss = -th.mean(score_fake)
        g_loss.backward()
        g_grad = np.mean([th.sum(p.grad **2).item() for p in itertools.chain(feature_model.parameters(),
                                                                 feature_model.parameters())])
    if (i_epoch % 10) != 0:
            optim_adv.step()
    else:
        optimizer.step()
        optim_dist.step()
    
    with th.no_grad():
        sample_wd_row = {}
        for i_class in range(len(inputs)):
            this_inputs = inputs[i_class]
            mean = means_per_cluster[i_class]
            log_std = stds_per_cluster[i_class]
            std = th.exp(log_std)
            y = np_to_var([i_class]).cuda()
            n_samples = len(this_inputs)
            samples = get_samples(n_samples, i_class)

            inverted = invert_hierarchical(samples)
            in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
            fake_np = var_to_np(inverted).reshape(len(inverted), -1)
            import ot

            dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
            match_matrix = ot.emd([],[], dist)
            cost = np.sum(dist * match_matrix)
            score_fake = adv_model(inverted, y)
            score_real = adv_model(inputs[i_class], y)
            wd_dist = th.mean(score_real) - th.mean(score_fake)
            sample_wd_row.update({
                'wd_dist_' + str(i_class): wd_dist.item() ,
                'sampled_wd' + str(i_class): cost,
                'wd_diff' + str(i_class): cost - wd_dist.item(),
            })
        end_time = time.time()
        epoch_row = {
        'd_loss': d_loss.item(),
        'g_loss': g_loss.item(),
        'grad_loss': gradient_loss.item(),
        'o_real': th.mean(score_real).item(),
        'o_fake': th.mean(score_fake).item(),
        'g_grad': g_grad,
        'd_grad': d_grad,
        'runtime': end_time -start_time,}
        epoch_row.update(sample_wd_row)
        df = df.append(epoch_row, ignore_index=True)
        if i_epoch % (max(1,n_epochs // 20)) == 0:
            display_text("Epoch {:d}".format(i_epoch))
            display(df.iloc[-5:])
        if i_epoch % (n_epochs // 20) == 0:
            print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))

            fig = plt.figure(figsize=(8,4))
            for i_class in range(len(inputs)):
                ins = var_to_np(inputs[i_class].squeeze())
                bps = np.abs(np.fft.rfft(ins.squeeze()))
                plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                n_samples = 5000
                samples = get_samples(n_samples, i_class)
                inverted = var_to_np(invert_hierarchical(samples).squeeze())
                bps = np.abs(np.fft.rfft(inverted.squeeze()))
                plt.plot(np.fft.rfftfreq(inverted.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0),
                        color=seaborn.color_palette()[i_class], ls='--')
            plt.title("Spectrum")
            plt.xlabel('Frequency [Hz]')

            plt.ylabel('Amplitude')
            plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
            display(fig)
            plt.close(fig)
            
            for i_class in range(len(inputs)):
                fig = plt.figure(figsize=(5,5))
                mean = means_per_cluster[i_class]
                log_std = stds_per_cluster[i_class]
                std = th.exp(log_std)
                y = np_to_var([i_class]).cuda()
                n_samples = 5000
                samples = get_samples(n_samples, i_class)
                inverted = var_to_np(invert_hierarchical(samples).squeeze())
                plt.plot(inverted.squeeze()[:,0], inverted.squeeze()[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class + 2], alpha=0.5, markersize=2)
                plt.plot(var_to_np(inputs[i_class].squeeze())[:,0], var_to_np(inputs[i_class].squeeze())[:,1],
                         ls='', marker='o', color=seaborn.color_palette()[i_class])

                display(fig)
                plt.close(fig)

In [ ]:
super_means_per_cluster

In [ ]:
th.exp(th.cat(super_stds_per_cluster))

In [ ]:
mean = means_per_cluster[0].clone()
mean.data[:4] += super_means_per_cluster[0].data[0]
mean.data[4:] += super_means_per_cluster[0].data[1]

examples = mean.repeat(20,1)

examples.data[:,:4] += th.linspace(-0.4,0.4,20).unsqueeze(1).cuda()

outs = amp_phase_sample_to_x_y(examples)
inverted = invert(feature_model, outs)


from matplotlib import rcParams, cycler
cmap = plt.cm.coolwarm
N = len(inverted)
with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
    plt.plot(var_to_np(inverted).squeeze().T);

In [ ]:
plt.plot(np.abs(np.fft.rfft(var_to_np(inverted.squeeze()))))

In [ ]:
plt.plot(np.angle(np.fft.rfft(var_to_np(this_inputs.squeeze()))))

In [ ]:
plt.plot(np.angle(np.fft.rfft(var_to_np(inverted.squeeze()))))

In [ ]:
mean = means_per_cluster[0].clone()
mean.data[:4] += super_means_per_cluster[0].data[0]
mean.data[4:] += super_means_per_cluster[0].data[1]

examples = mean.repeat(20,1)

examples.data[:,4:] += th.linspace(-0.5,0.5,20).unsqueeze(1).cuda()

outs = amp_phase_sample_to_x_y(examples)
inverted = invert(feature_model, outs)


from matplotlib import rcParams, cycler
cmap = plt.cm.coolwarm
N = len(inverted)
with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
    plt.plot(var_to_np(inverted).squeeze().T);

In [ ]:
plt.plot(np.angle(np.fft.rfft(var_to_np(inverted.squeeze()))))

In [ ]:
plt.plot(np.abs(np.fft.rfft(var_to_np(inverted.squeeze()))))

In [ ]:
th.exp(th.cat(stds_per_cluster))

In [ ]:
subsamples

In [ ]:
# std is sqrt of sum of squares

In [ ]:
th.sum(std ** 2)